#Materiały

https://github.com/databricks/LearningSparkV2

#Setup

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=607fa99283f8c7181e00023c81f7a37f5acc982afc87031dfd848190fc8b03bb
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, DateType, TimestampType
from pyspark.sql.functions import (
    col, size, lit, explode,
    concat, concat_ws, substring,
    datediff, date_add, date_sub, date_format,
    year, month, dayofmonth, dayofweek, dayofyear, weekofyear,
    hour, minute, second, to_timestamp,
    count, countDistinct, min, max, avg, sum, udf, when,
    desc, asc

)

# import pyspark.sql.functions as F

from datetime import datetime

In [5]:
spark = SparkSession.builder.appName("a").getOrCreate()

In [6]:
spark

#002A. Zliczanie cukierków M&M's

###002A_gen_mnm_dataset.py

In [ ]:
import sys
import random
import csv

def get_random_choice(lst):
  return random.choice(lst)

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: gen_mnm_dataset entries", file=sys.stderr)
        sys.exit(-1)

    states = ["CA", "WA", "TX", "NV", "CO", "OR", "AZ", "WY", "NM", "UT"]
    colors = ["Brown", "Blue", "Orange", "Yellow", "Green", "Red"]
    fieldnames = ['State', 'Color', 'Count']


    entries = int(sys.argv[1])
    dataset_fn = "mnm_dataset.csv"

    with open(dataset_fn, mode='w') as dataset_file:
        dataset_writer = csv.writer(dataset_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        dataset_writer.writerow(fieldnames)
        for i in range(1, entries):
            dataset_writer.writerow([get_random_choice(states), get_random_choice(colors), random.randint(10, 100)])
    print("Wrote %d lines in %s file" % (entries, dataset_fn))

Usage: gen_mnm_dataset entries


SystemExit: -1

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# python3 002A_gen_mnm_dataset.py 1000 # 002A_gen_mnm_dataset.py pierwszy argv + 1000 drugi argv

### 002A_gen_mnm_dataset.py (Colab)

In [ ]:
import random
import csv

def get_random_choice(lst):
    return random.choice(lst)

if __name__ == "__main__":
    entries = 100000

    states = ["CA", "WA", "TX", "NV", "CO", "OR", "AZ", "WY", "NM", "UT"]
    colors = ["Brown", "Blue", "Orange", "Yellow", "Green", "Red"]
    fieldnames = ['State', 'Color', 'Count']

    dataset_fn = "mnm_dataset.csv"

    with open(dataset_fn, mode='w') as dataset_file:
        dataset_writer = csv.writer(dataset_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        dataset_writer.writerow(fieldnames)
        for i in range(1, entries):
            dataset_writer.writerow([get_random_choice(states), get_random_choice(colors), random.randint(10, 100)])
    print("Wrote %d lines in %s file" % (entries, dataset_fn))

Wrote 100000 lines in mnm_dataset.csv file


###test_agg.py


In [ ]:
from __future__ import print_function

import sys

from pyspark.sql import SparkSession

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: mnmcount <file>", file=sys.stderr)
        sys.exit(-1)

    spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())
    # get the M&M data set file name
    mnm_file = sys.argv[1]
    # read the file into a Spark DataFrame
    mnm_df = (spark.read.format("csv")
                .option("header", "true")
                .option("inferSchema", "true")
                .load(mnm_file))
    mnm_df.show(n=10, truncate=False)
    # aggregate count of all colors and groupBy state and color
    # orderBy descending order
    count_mnm_df = (mnm_df.select("State", "Color", "Count")
                    .groupBy("State", "Color")
                    .sum("Count")
                    .orderBy("sum(Count)", ascending=False))

    count_mnm_df.show()

In [ ]:
# python3 test_agg.py /content/mnm_dataset.csv

###test_agg.py (Colab)

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("a")
         .getOrCreate())

mnm_file = "/content/mnm_dataset.csv"

mnm_df = (spark.read.format("csv")
          .option("header", "true")
          .option("inferSchema", "true")  # Automatycznie rozpoznaj typy danych
          .load(mnm_file))

mnm_df.show(n=10, truncate=False)
mnm_df.printSchema()

count_mnm_df = (mnm_df.select("State", "Color", "Count")
                .groupBy("State", "Color")
                .sum("Count")
                .orderBy("sum(Count)", ascending=False))

count_mnm_df.show(n=30)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|NV   |Red   |21   |
|CO   |Orange|86   |
|TX   |Yellow|54   |
|NV   |Yellow|72   |
|WA   |Brown |68   |
|OR   |Blue  |64   |
|NV   |Yellow|44   |
|OR   |Yellow|32   |
|CO   |Red   |10   |
|OR   |Yellow|95   |
+-----+------+-----+
only showing top 10 rows

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   UT|   Red|     96570|
|   AZ|Yellow|     96194|
|   UT|Orange|     96132|
|   TX|Orange|     96117|
|   WA| Brown|     95920|
|   OR|  Blue|     95539|
|   TX| Green|     94796|
|   CO|   Red|     94338|
|   UT| Green|     94166|
|   CA| Green|     93947|
|   TX|Yellow|     93745|
|   NM| Green|     93594|
|   CO| Brown|     93503|
|   NM|   Red|     93367|
|   OR|Yellow|     93257|
|   CA|Yellow|     93079|
|   WY| Brown|     93050|
|   WY|   Red|     92869|
|   OR|   Red|  

###mnmcount.py (Colab)

In [ ]:
from __future__ import print_function
import sys
from pyspark.sql import SparkSession

spark = (SparkSession
    .builder
    .appName("PythonMnMCount")
    .getOrCreate())

mnm_file = "/content/mnm_dataset.csv"

mnm_df = (spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load(mnm_file))

mnm_df.show(n=5, truncate=False)

count_mnm_df = (mnm_df.select("State", "Color", "Count")
                .groupBy("State", "Color")
                .sum("Count")
                .orderBy("sum(Count)", ascending=False))

count_mnm_df.show(n=30, truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

ca_count_mnm_df = (mnm_df.select("*")
                   .where(mnm_df.State == 'CA')
                   .groupBy("State", "Color")
                   .sum("Count")
                   .orderBy("sum(Count)", ascending=False))

ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|NV   |Red   |21   |
|CO   |Orange|86   |
|TX   |Yellow|54   |
|NV   |Yellow|72   |
|WA   |Brown |68   |
+-----+------+-----+
only showing top 5 rows

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|UT   |Red   |96570     |
|AZ   |Yellow|96194     |
|UT   |Orange|96132     |
|TX   |Orange|96117     |
|WA   |Brown |95920     |
|OR   |Blue  |95539     |
|TX   |Green |94796     |
|CO   |Red   |94338     |
|UT   |Green |94166     |
|CA   |Green |93947     |
|TX   |Yellow|93745     |
|NM   |Green |93594     |
|CO   |Brown |93503     |
|NM   |Red   |93367     |
|OR   |Yellow|93257     |
|CA   |Yellow|93079     |
|WY   |Brown |93050     |
|WY   |Red   |92869     |
|OR   |Red   |92838     |
|CA   |Blue  |92670     |
|OR   |Brown |92659     |
|OR   |Orange|92626     |
|NV   |Green |92563     |
|CA   |Brown |92545     |
|AZ   |Red   |92476     |
|NV   |Orange|92440     |
|WA   |Orange|92210     |
|NV   

#002B. API Strukturalne : DataFrame (Fire)

###002B_example-3_6.py

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]

if __name__ == "__main__":

   spark = (SparkSession
       .builder
       .appName("a")
       .getOrCreate())

   blogs_df = spark.createDataFrame(data, schema)
   blogs_df.show(truncate=False)
   print()

   blogs_df.printSchema()

    # expr() (expression) pozwala pisać wyrażenia SQL-owe w PySpark. W tym przypadku wyrażenie SQL to po prostu kolumna „Hits”.
   blogs_df.select(expr("Hits") * 2).show(2)
   blogs_df.select(col("Hits") * 2).show(2)
    # Ponownie używamy funkcji expr(), jednak tutaj całe wyrażenie „Hits * 2” jest traktowane jako wyrażenie SQL-owe.
   blogs_df.select(expr("Hits * 2")).show(2)

   blogs_df.withColumn("Big Hitters", (expr("Hits > 10000"))).show(truncate=False)
   blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+----------------------------+
|Id |First    |Last   |Url              |Published|Hits |Campaigns                   |
+---+---------+-------+-----------------+---------+-----+----------------------------+
|1  |Jules    |Damji  |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke   |Wenig  |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny    |Lee    |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata|Das    |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei    |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold  |Xin    |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+---------+-------+-----------------+---------+-----+----------------------------+


root
 |-- Id: integer (nullable = false)
 |-- First: string (nullable = false)
 |-- Last: string (nullable = false)
 |-- Url: st

###002B_json_example-3_6.py

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

if __name__ == "__main__":

   spark = (SparkSession
       .builder
       .appName("JSONDataExample")
       .getOrCreate())

   json_file = "/content/002B_blogs.json"

   blogs_df = spark.read.schema(schema).json(json_file)

   blogs_df.show(truncate=False)
   print()

   blogs_df.printSchema()

   blogs_df.select(expr("Hits") * 2).show(2)
   blogs_df.select(col("Hits") * 2).show(2)
   blogs_df.select(expr("Hits * 2")).show(2)

   blogs_df.withColumn("Big Hitters", expr("Hits > 10000")).show(truncate=False)

   blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+----------------------------+
|Id |First    |Last   |Url              |Published|Hits |Campaigns                   |
+---+---------+-------+-----------------+---------+-----+----------------------------+
|1  |Jules    |Damji  |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke   |Wenig  |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny    |Lee    |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata|Das    |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei    |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold  |Xin    |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+---------+-------+-----------------+---------+-----+----------------------------+


root
 |-- Id: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- Url: strin

In [ ]:
# Sortowanie wynikowego DataFrame po kolumnie "Id" w kolejności rosnącej
blogs_df.orderBy("Id").show(truncate=False)

# Lub można również użyć funkcji sort() (jest to równoważne)
blogs_df.sort("Id").show(truncate=False)

# Sortowanie w kolejności malejącej
blogs_df.orderBy(col("Id").desc()).show(truncate=False)

+---+---------+-------+-----------------+---------+-----+----------------------------+
|Id |First    |Last   |Url              |Published|Hits |Campaigns                   |
+---+---------+-------+-----------------+---------+-----+----------------------------+
|1  |Jules    |Damji  |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke   |Wenig  |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny    |Lee    |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata|Das    |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei    |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold  |Xin    |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+---------+-------+-----------------+---------+-----+----------------------------+

+---+---------+-------+-----------------+---------+-----+----------------------------+
|Id |First    |Last   |Url              |P

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import *

spark = (SparkSession
        .builder
	.appName("Authors")
	.getOrCreate())

schema = StructType([
	StructField("Author" , StringType(), False),
	StructField("State", StringType(), False)])

rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, schema)
authors_df.show()

+-------------+-----+
|       Author|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



###002B_sf-fire-calls.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

schema = StructType([
    StructField("CallNumber", IntegerType(), False),
    StructField("UnitID", StringType(), False),
    StructField("IncidentNumber", IntegerType(), False),
    StructField("CallType", StringType(), True),
    StructField("CallDate", StringType(), True),
    StructField("WatchDate", StringType(), True),
    StructField("CallFinalDisposition", StringType(), True),
    StructField("AvailableDtTm", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Zipcode", IntegerType(), True),
    StructField("Battalion", StringType(), True),
    StructField("StationArea", StringType(), True),
    StructField("Box", StringType(), True),
    StructField("OriginalPriority", IntegerType(), True),
    StructField("Priority", IntegerType(), True),
    StructField("FinalPriority", IntegerType(), True),
    StructField("ALSUnit", BooleanType(), True),
    StructField("CallTypeGroup", StringType(), True),
    StructField("NumAlarms", IntegerType(), True),
    StructField("UnitType", StringType(), True),
    StructField("UnitSequenceInCallDispatch", IntegerType(), True),
    StructField("FirePreventionDistrict", IntegerType(), True),
    StructField("SupervisorDistrict", IntegerType(), True),
    StructField("Neighborhood", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("RowID", StringType(), True),
    StructField("Delay", FloatType(), True)
])

spark = SparkSession.builder.appName("FireCalls").getOrCreate()

file_path = "/content/002B_sf-fire-calls.csv"
fire_df = spark.read.csv(file_path, header=True, schema=schema)

fire_df.show(5, truncate=False)

fire_df.printSchema()

+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+---------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood         |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+--

In [ ]:
output_path = "/content/parquet"

fire_df.write.parquet(output_path)

In [ ]:
few_fire_df = (fire_df
    .select("IncidentNumber", "AvailableDtTm", "CallType")
    .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [ ]:
# method chaining

(fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .agg(countDistinct("CallType").alias("DistinctCallTypes"))
    .show())

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [ ]:
fire_df.select("CallType") \
    .where(col("CallType").isNotNull()) \
    .agg(countDistinct("CallType") \
    .alias("DistinctCallTypes")) \
    .show()

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [ ]:
(fire_df
      .select("CallType")
      .where(col("CallType").isNotNull())
      .distinct()
      .show(10, False)
)

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



In [ ]:
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")

(new_fire_df
    .select("ResponseDelayedinMins")
    .where(col("ResponseDelayedinMins") > 5)
    .orderBy(col("ResponseDelayedinMins").desc())
    .show(10, False)
)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|1844.55              |
|1370.2333            |
|949.11664            |
|931.45               |
|751.93335            |
|734.05               |
|721.4                |
|628.61664            |
|535.93335            |
|491.26666            |
+---------------------+
only showing top 10 rows



In [ ]:
fire_ts_df = (new_fire_df
      .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
      .drop("CallDate")
      .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
      .drop("WatchDate")
      .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
      .drop("AvailableDtTm"))

(fire_ts_df
    .select("IncidentDate", "OnWatchDate", "AvailableDtTS")
    .show(5, False))

fire_ts_df.printSchema()

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable =

In [ ]:
(fire_ts_df
    .select(year('IncidentDate').alias('Year'))
    .distinct()
    .orderBy('Year')
    .show())

+----+
|Year|
+----+
|2000|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2007|
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
+----+



In [ ]:
(fire_ts_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .groupBy("CallType")
    .count()
    .orderBy("count", ascending=False)
    .show(n=10, truncate=False))

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [ ]:
(fire_ts_df
    .select(
        sum("NumAlarms"),
        avg("ResponseDelayedinMins"),
        min("ResponseDelayedinMins"),
        max("ResponseDelayedinMins")
    )
    .show())

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|         3.892364154521585|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



In [ ]:
fire_ts_df.explain(True)

== Parsed Logical Plan ==
Project [CallNumber#0, UnitID#1, IncidentNumber#2, CallType#3, CallFinalDisposition#6, Address#8, City#9, Zipcode#10, Battalion#11, StationArea#12, Box#13, OriginalPriority#14, Priority#15, FinalPriority#16, ALSUnit#17, CallTypeGroup#18, NumAlarms#19, UnitType#20, UnitSequenceInCallDispatch#21, FirePreventionDistrict#22, SupervisorDistrict#23, Neighborhood#24, Location#25, RowID#26, ... 4 more fields]
+- Project [CallNumber#0, UnitID#1, IncidentNumber#2, CallType#3, CallFinalDisposition#6, AvailableDtTm#7, Address#8, City#9, Zipcode#10, Battalion#11, StationArea#12, Box#13, OriginalPriority#14, Priority#15, FinalPriority#16, ALSUnit#17, CallTypeGroup#18, NumAlarms#19, UnitType#20, UnitSequenceInCallDispatch#21, FirePreventionDistrict#22, SupervisorDistrict#23, Neighborhood#24, Location#25, ... 5 more fields]
   +- Project [CallNumber#0, UnitID#1, IncidentNumber#2, CallType#3, CallFinalDisposition#6, AvailableDtTm#7, Address#8, City#9, Zipcode#10, Battalion#11,

In [ ]:
from pyspark.sql import Row

row = Row(350, True, "Learning Spark 2E", None)

print(row[0])
print(row[1])
print(row[2])
print(row[3])

350
True
Learning Spark 2E
None


#002C. Spark SQL (departuredelays)

In [ ]:
spark = (SparkSession
         .builder
         .appName("SparkSchemaExample")
         .getOrCreate())

schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

csv_file = "/content/002C_departuredelays.csv"

df = (spark.read.format("csv")
      .option("header", "true")
      .schema(schema)
      .load(csv_file))

df.show(5)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [ ]:
spark = (SparkSession
         .builder
         .appName("SparkSQLExampleApp")
         .getOrCreate())

csv_file = "/content/002C_departuredelays.csv"

df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")

result_df = spark.sql("SELECT * FROM us_delay_flights_tbl WHERE delay > 1000")

result_df.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1042359| 1033|    2090|   ANC|        DEN|
|1180805| 1560|     548|   BNA|        DFW|
|1101330| 1180|     822|   BOS|        ATL|
|1211400| 1288|    1057|   BWI|        DFW|
|1011915| 1191|     565|   CLT|        MIA|
|1051805| 1461|    1734|   CMH|        LAX|
|1021720| 1321|     705|   CVG|        DFW|
|1011155| 1174|     557|   DEN|        DFW|
|1062145| 1054|     549|   DTW|        BOS|
|1300725| 1482|     626|   EGE|        DFW|
|1300915| 1500|    1517|   EGE|        JFK|
|1041440| 1159|     589|   GUC|        DFW|
|1311800| 1004|    3288|   HNL|        DFW|
|1210725| 1444|    1018|   IAD|        DFW|
|1301345| 1170|    1518|   LAS|        ATL|
|1011700| 1017|     887|   MSP|        LGA|
|1150715| 1496|    1033|   ONT|        DFW|
|1071715| 1149|    1040|   ORD|        MIA|
|1181225| 1151|     994|   PBI|        ORD|
|1141620| 1177|    1132|   PHL| 

In [ ]:
result_df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



###Spark SQL vs API DataFrame

In [ ]:
spark.sql("""
    SELECT distance, origin, destination
    FROM us_delay_flights_tbl
    WHERE distance > 1000
    ORDER BY distance DESC
""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [ ]:
(df.select("distance", "origin", "destination")
   .where(col("distance") > 1000)
   .orderBy(desc("distance"))
   .show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [ ]:
(df.select("distance", "origin", "destination")
   .where("distance > 1000")
   .orderBy("distance", ascending=False)
   .show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [ ]:
spark.sql("""
    SELECT date, delay, origin, destination
    FROM us_delay_flights_tbl
    WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
    ORDER BY delay DESC
""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [ ]:
formatted_df = spark.sql("""
    SELECT
        date,
        delay,
        origin,
        destination,
        -- Przekształcenie liczby na string, wypełnienie zerami z lewej strony do 8 znaków, a następnie konwersja na timestamp
        date_format(to_timestamp(lpad(date, 8, '0'), 'MMddHHmm'), 'MM-dd hh:mm a') AS formatted_date
    FROM us_delay_flights_tbl
    WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
    ORDER BY delay DESC
""")

# Wyświetlenie wyników
formatted_df.select("formatted_date", "delay", "origin", "destination").show(10, truncate=False)

+--------------+-----+------+-----------+
|formatted_date|delay|origin|destination|
+--------------+-----+------+-----------+
|02-19 09:25 AM|1638 |SFO   |ORD        |
|01-03 05:55 PM|396  |SFO   |ORD        |
|01-02 11:30 PM|326  |SFO   |ORD        |
|01-05 12:05 PM|320  |SFO   |ORD        |
|01-19 09:25 AM|297  |SFO   |ORD        |
|02-17 11:15 AM|296  |SFO   |ORD        |
|01-07 10:40 AM|279  |SFO   |ORD        |
|01-05 03:50 PM|274  |SFO   |ORD        |
|03-12 07:30 AM|266  |SFO   |ORD        |
|01-26 11:04 AM|258  |SFO   |ORD        |
+--------------+-----+------+-----------+
only showing top 10 rows



In [ ]:
spark.sql("""
    SELECT
        delay,
        origin,
        destination,
        CASE
            WHEN delay > 360 THEN 'Very Long Delays'
            WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
            WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
            WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delays'
            WHEN delay = 0 THEN 'No Delays'
            ELSE 'Early'
        END AS Flight_Delays
    FROM us_delay_flights_tbl
    ORDER BY origin, delay DESC
""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



###Utworzenie bazy danych

In [ ]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

####Utworzenie tabeli niezarządzanej

Natomiast w przypadku tabeli niezarządzanej Spark zarządza jedynie metadanymi, a ja — danymi.

In [ ]:
spark.sql("""
    CREATE TABLE unmanaged_us_delay_flights_tbl(
        date STRING,
        delay INT,
        distance INT,
        origin STRING,
        destination STRING
    )
    USING csv
    OPTIONS (PATH '/content/002C_departuredelays.csv')
""")

DataFrame[]

In [ ]:
(df
    .write
    .option("path", "/content/unmanaged")
    .saveAsTable("unmanaged_us_delay_flights_tbl_df"))

####Utworzenie tabeli zarządzanej

W przypadku tabeli zarządzanej framework zarządza metadanymi i danymi znajdującymi się w pliku magazynu danych. Może to być lokalny system plików, HDFS lub obiekt magazynu danych, takiego jak Amazon S3 lub Azure Blob.

In [ ]:
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

In [ ]:
csv_file = "/content/002C_departuredelays.csv"

schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"

flights_df = spark.read.csv(csv_file, schema=schema)

flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

# /content/spark-warehouse/learn_spark_db.db/managed_us_delay_flights_tbl/part-00001-3f9b8262-8cf0-48a3-b35a-3e0257f14acb-c000.snappy.parquet

###Tworzenie widoku

Widoki można podzielić na globalne (dostępne we wszystkich egzemplarzach SparkSession w danym klastrze) oraz mające zasięg sesji (dostępne jedynie dla pojedynczego egzemplarza SparkSession).

In [ ]:
CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
    SELECT date, delay, origin, destination
    FROM us_delay_flights_tbl
    WHERE origin = 'SFO';

In [ ]:
CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
    SELECT date, delay, origin, destination
    FROM us_delay_flights_tbl
    WHERE origin = 'JFK';

In [ ]:
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")

In [ ]:
df_sfo.show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
+-------+-----+------+-----------+
only showing top 5 rows



In [ ]:
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")

In [ ]:
df_jfk.show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



###Wyświetlanie metadanych

In [ ]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/content/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/content/spark-warehouse/learn_spark_db.db')]

In [ ]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='unmanaged_us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='unmanaged_us_delay_flights_tbl_df', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

###Wczytywanie zawartości tabeli do egzemplarza DataFrame

In [ ]:
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")

us_flights_df2 = spark.table("us_delay_flights_tbl")

In [ ]:
us_flights_df2.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



###Źródła danych dla egzemplarzy DataFrame i tabel SQL

####Parquet

In [ ]:
file = "/content/parquet"

df = spark.read.format("parquet").load(file)

df.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [ ]:
# Tworzenie tymczasowego widoku na podstawie danych z pliku Parquet
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING parquet
    OPTIONS (
        path "/content/parquet"
    )
""")

# Wywołanie zapytania SQL do przeglądania danych z tymczasowego widoku
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [ ]:
(df.write.format("parquet")
    .mode("overwrite")  # Nadpisuje dane, jeśli już istnieją
    .option("compression", "snappy")  # Ustawia kompresję na Snappy
    .save("/content/parquet"))  # Ścieżka, gdzie dane zostaną zapisane

In [ ]:
# Zapis DataFrame jako tabeli w Spark SQL
(df.write
    .mode("overwrite")  # Nadpisuje istniejącą tabelę, jeśli już istnieje
    .saveAsTable("us_delay_flights_tbl"))  # Zapisuje dane jako tabelę o nazwie "us_delay_flights_tbl"

####JSON

In [ ]:
# Ścieżka do pliku JSON
file = "/content/json/*"

# Odczyt danych z pliku JSON i załadowanie do DataFrame
df = spark.read.format("json").load(file)

In [ ]:
# Tworzenie lub zastąpienie tymczasowego widoku z użyciem danych JSON

%%sql

CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING json
OPTIONS (
    path "/content/json/*"
)

UsageError: Cell magic `%%sql` not found.


In [ ]:
df.createOrReplaceTempView("json")
df.show(10)

# Wykonanie zapytania SQL na widoku
spark.sql("SELECT * FROM json").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    Uni

In [ ]:
# Zapisanie DataFrame w formacie JSON z kompresją Snappy
df.write.format("json") \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .save("/content/data/json/df_json")

In [ ]:
!ls /content/data/json/df_json

part-00000-2b0e6f40-2617-4ca5-81e0-c7ab94178522-c000.json.snappy  _SUCCESS
part-00001-2b0e6f40-2617-4ca5-81e0-c7ab94178522-c000.json.snappy


####CSV

In [ ]:
file = "/content/scv*"

schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

df = (spark.read.format("csv")
      .option("header", "true")
      .schema(schema)
      .option("mode", "FAILFAST")  # Zakończenie w przypadku błędów
      .option("nullValue", "")     # Zastąpienie brakujących wartości pustym ciągiem
      .load(file))

df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [ ]:
# Utworzenie tymczasowego widoku
df.createOrReplaceTempView("csv")

# Sprawdzenie danych w widoku
spark.sql("SELECT * FROM csv LIMIT 10").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+



In [ ]:
# Zapisanie DataFrame w formacie CSV
(df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("/content/scv/df_csv"))

In [ ]:
!ls /content/scv/df_csv

part-00000-7b53ab58-00f8-40f8-8380-0a52a96a21a2-c000.csv  _SUCCESS
part-00001-7b53ab58-00f8-40f8-8380-0a52a96a21a2-c000.csv


####Avro

Jest używany np. przez Apache Kafka do serializacji i deserializacji komunikatów. Format ten posiada wiele zalet, m.in. bezpośrednie mapowanie na JSON, dużą szybkość działania i efektywność. Jego obsługę zapewniono w wielu językach programowania.


In [9]:
# df = (spark.read.format("avro")
#     .load("/content/avro/*"))

# df.show(truncate=False)

In [ ]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW avro
USING avro
OPTIONS (
  path "/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
)

In [ ]:
spark.sql("SELECT * FROM avro").show(truncate=False)

In [ ]:
(df.write
  .format("avro")
  .mode("overwrite")
  .save("/tmp/data/avro/df_avro"))

####ORC

In [ ]:
# /content/part-r-00000-2c4f7d96-e703-4de3-af1b-1441d172c80f.snappy.orc

In [12]:
file = "/content/part-r-00000-2c4f7d96-e703-4de3-af1b-1441d172c80f.snappy.orc"

df = spark.read.format("orc").option("path", file).load()

df.show(10, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [ ]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING orc
OPTIONS (
  path "/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
)

In [13]:
df.createOrReplaceTempView("orc")

In [14]:
# Wykonanie zapytania SQL na tymczasowym widoku i wyświetlenie wyników
spark.sql("SELECT * FROM orc").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [15]:
(df.write.format("orc")
  .mode("overwrite")
  .option("compression", "snappy")
  .save("/content/orc/"))

####Obrazy

In [16]:
from pyspark.ml import image

image_dir = "/content/train_images"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)



####Pliki binarne